## OBJETIVOS:
-Se nos planteó el problema de una web que quería realizar una clasificación rápida de libros mediante un modelo. 
Por lo que, a modo de presentación de un primer borrador nos marcamos como objetivo entregar una solución rápida
que mostrase el funcionamiento para futuros resultados.

-Generación de los datos: para poder hacer el modelo necesitamos generar unos datos, por lo que accedimos a una web
y obtuvimos manualmente las urls de los libros y nos creamos un fichero excel además con el índice de nuestro 
problema multietiqueta. Una vez hecho esto, creamos un cuaderno(generar_csv.ipynb) para generar nuestro csv.

-Con los datos obtenidos, realizamos un proceso de NLP, con su limpieza y todas sus características. Para mayor
entendimiento del proceso realizado seguir leyendo el siguiente resumen o echar un vistazo al artículo que se 
encuentra también en este repositorio.

En este caso tenemos ocho géneros: humor, amor, ciencia ficción, aventuras, histórico,
psicológico, guerra y terror.

Ahora pasamos a explicar la estructura que hemos seguido en el proyecto.

## Estructura:
En primer lugar, accedimos a la web proporcionada y creamos un excel  con las urls de
los libros y las categorías a la que pertenecían.
Una vez hecho esto creamos un cuaderno el cual accedía las urls y descargaba el texto
creando un nuevo dataset.(generar_csv.ipynb)
Luego creamos ya nuestro cuaderno para el trabajo en cual ya preprocesamos los datos 
y los limpiamos para dejarlos preparados. Después probamos diferentes estrategias de 
resolución aplicando TFIDF(la mejor opción), word2vectors( el cual nos dió problemas y 
tuvimos que descartar ya que los libros contenían demasiadas palabras) y redes 
neuronales(las cuales apartamos pues no conseguíamos crear una estructura que encontrase
un modelo más o menos fiable para su optimización).

Al final optamos por la estrategia de optimizar
TFIDF primero, luego aplicar PCA y finalmente optimizar el algoritmo Linear SVC.



## ENFOQUE FUTURO:
- Respecto a este proyecto, decir que no está completo y se va a seguir trabajando
en él pero de manera privada por confidencialidad.
- Para mejorar este primer borrador debemos hacer un dataset muchísimo más
amplío pues es súper clave en estos casos de multietiquetas. Y una vez hecho esto, optimizar
el proyecto con diferentes estrategias. Aquí solo usé una pues se trata de un borrador inicial
y quería que se viese el progreso del trabajo.

In [ ]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-dzifqja_
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-dzifqja_
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=11756 sha256=3bb61fd9b1a4e1a7eaf70e1454a47faa17c92ff8a8efdbe2bef8aa0e67bcceba
  Stored in directory: /tmp/pip-ephem-wheel-cache-i7bxloij/wheels/0d/b3/29/bfe3deffda68980088d17b81331be6667e837ffb4a071bae82
Successfully built preprocess-kgptalkie


In [ ]:
!python -m spacy download en_core_web_lg

2022-10-28 06:09:05.423428: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 15 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
#IMPORTS
import pandas as pd
import numpy as np
import requests

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import preprocess_kgptalkie as kgp
from spacy.lang.en.stop_words import STOP_WORDS as stopwords

from sklearn.metrics import plot_confusion_matrix,classification_report
import tensorflow as tf

from tensorflow import keras
from keras.callbacks import TensorBoard

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import spacy
import pickle as pkl

In [ ]:
%load_ext tensorboard

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/libros/prueba_final.csv")

In [ ]:
df.head()

,Unnamed: 0,url,target,text,clean_text
0,0,https://www.gutenberg.org/cache/epub/68468/pg6...,[0],the project gutenberg ebook of told by the col...,project gutenberg ebook told colonel william l...
1,1,https://www.gutenberg.org/cache/epub/63429/pg6...,"[0,3]",the project gutenberg ebook of joe carsons wea...,project gutenberg ebook joe carsons weapon jam...
2,2,https://www.gutenberg.org/cache/epub/15247/pg1...,[0],the project gutenberg ebook of john henry smit...,project gutenberg ebook john henry smith frede...
3,3,https://www.gutenberg.org/cache/epub/47087/pg4...,[4],the project gutenberg ebook of motor matts rac...,project gutenberg ebook motor matts race stanl...
4,4,https://www.gutenberg.org/cache/epub/18857/pg1...,"[3,4]",project gutenbergs a journey to the centre of ...,project gutenbergs journey centre earth jules ...


In [ ]:
data=pd.DataFrame()
data["text"]=df["clean_text"]
data["target"]=df["target"]
data.head()

,text,target
0,project gutenberg ebook told colonel william l...,[0]
1,project gutenberg ebook joe carsons weapon jam...,"[0,3]"
2,project gutenberg ebook john henry smith frede...,[0]
3,project gutenberg ebook motor matts race stanl...,[4]
4,project gutenbergs journey centre earth jules ...,"[3,4]"


In [ ]:
#TRATAMIENTO DEL TARGET
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(data['target'])
classes = multilabel.classes_
y_=pd.DataFrame(y,columns=classes)
y_.head()

,",",0,1,3,4,5,6,7,8,[,]
0,0,1,0,0,0,0,0,0,0,1,1
1,1,1,0,1,0,0,0,0,0,1,1
2,0,1,0,0,0,0,0,0,0,1,1
3,0,0,0,0,1,0,0,0,0,1,1
4,1,0,0,1,1,0,0,0,0,1,1


In [ ]:
y_=y_.drop([",","[","]"], axis=1)
y_.columns=["humor","amor","science fiction","aventuras","historica","psicologico","guerra","terror"]
y_.head()

,humor,amor,science fiction,aventuras,historica,psicologico,guerra,terror
0,1,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0
4,0,0,1,1,0,0,0,0


In [ ]:
y2_=np.array(y_)
y2_.shape

(370, 8)

# TFIDF + PCA +CLASIFICADOR

In [ ]:
x1=data["text"]
y1=y2_

tfidf=TfidfVectorizer()
x1=tfidf.fit_transform(x1)

print("shape of x1", x1.shape)
print("shape of y1", y1.shape)

shape of x1 (370, 428732)
shape of y1 (370, 8)


In [ ]:
# COMPROBACIÓN DEL USO DE PCA
x1_=x1.toarray()
x1_=pd.DataFrame(x1_)

In [ ]:
#Con esto código probamos la varianza explicativa para cada número de variables escogidos
min_var = 0.9
pca_var= PCA(n_components=min_var, svd_solver='full')

X_pca = pca_var.fit_transform(x1_)

X_pca.shape

pca_var= PCA(n_components=min_var, svd_solver='full')

X_pca2 = pca_var.fit(x1_)

print('Nº de componentes:', len(X_pca2.components_))
print('Ratio de varianza explicada:')
print(X_pca2.explained_variance_ratio_)

Nº de componentes: 270
Ratio de varianza explicada:
[0.03755544 0.01975315 0.01419666 0.01162174 0.01089178 0.0090699
 0.00845298 0.00723166 0.00691701 0.00652449 0.00640314 0.00635176
 0.00609314 0.00605043 0.00587647 0.00581999 0.00571814 0.00569693
 0.00566647 0.00559701 0.00555345 0.00547821 0.00545874 0.00533067
 0.00529968 0.00525467 0.00517536 0.00506356 0.00505453 0.00496267
 0.00492757 0.00482235 0.00475257 0.00469301 0.00458135 0.00447935
 0.00430926 0.00422373 0.00419836 0.00414729 0.00409113 0.0040253
 0.00399901 0.00396558 0.00392798 0.00386911 0.00383957 0.00379471
 0.00377625 0.00371433 0.00370717 0.00364975 0.0035667  0.00354873
 0.00353494 0.00351748 0.00348052 0.0034267  0.00339137 0.00335126
 0.00333218 0.00331694 0.00329674 0.00328665 0.00324556 0.003242
 0.00320862 0.00318384 0.00317369 0.00314912 0.00314232 0.00312602
 0.00311846 0.00310718 0.00309464 0.00307671 0.00306382 0.00305763
 0.00303767 0.00303297 0.00302886 0.00301506 0.00301056 0.00300288
 0.0029877  0.

Somos capaces de explicar un 90% de la varianza del dataset reduciendo nuestros atributos de 428.732 a 270. Es prácticamente obligatorio utilizar PCA en este proyecto.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_pca, y_, test_size = 0.2, random_state = 0)

Una vez tenemos los datos en una forma correcta, vamos a realizar varias formas de solucionar nuestro proyecto y finalmente seleccionaremos el que mejores resultados nos proporcione para después optimizarlo.

* OneVsRestClassifier

## OneVsRestClassifier

In [ ]:
def j_score(y_true, y_pred):
    jaccard = np.minimum(y_true, y_pred).sum(axis = 1)/np.maximum(y_true, y_pred).sum(axis = 1)
    return jaccard.mean()*100

In [ ]:
lr = LogisticRegression(solver='lbfgs')
clf = OneVsRestClassifier(lr)
clf.fit(x_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [ ]:
y_pred = clf.predict(x_test)
j_score(y_test, y_pred)

3.3783783783783785

## SVM

In [ ]:
from sklearn.svm import LinearSVC
svm = LinearSVC(C = 1.5, penalty='l1', dual = False)
clf = OneVsRestClassifier(svm)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
j_score(y_test, y_pred)

39.86486486486486

## Random Forest

In [ ]:
model = RandomForestClassifier()
model.fit(x_train,y_train)
prediction_forest = model.predict(x_test)

In [ ]:
report = classification_report(y_test, prediction_forest)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.30      0.46        10
           1       0.86      0.55      0.67        11
           2       1.00      0.36      0.53        14
           3       1.00      0.12      0.21        17
           4       1.00      0.25      0.40        20
           5       1.00      0.31      0.47        13
           6       1.00      0.10      0.18        10
           7       0.00      0.00      0.00         7

   micro avg       0.96      0.25      0.40       102
   macro avg       0.86      0.25      0.36       102
weighted avg       0.92      0.25      0.38       102
 samples avg       0.22      0.20      0.21       102



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_pred = model.predict(x_test)
j_score(y_test, y_pred)

20.045045045045043

## KNN

In [ ]:
knn=KNeighborsClassifier()
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)
j_score(y_test, y_pred)

8.108108108108109

# Word2vec

In [ ]:
data.head()

,text,target
0,project gutenberg ebook told colonel william l...,[0]
1,project gutenberg ebook joe carsons weapon jam...,"[0,3]"
2,project gutenberg ebook john henry smith frede...,[0]
3,project gutenberg ebook motor matts race stanl...,[4]
4,project gutenbergs journey centre earth jules ...,"[3,4]"


In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
def get_vec(x):
    doc = nlp(x)
    vec = doc.vector
    return vec

In [ ]:
#df['vectors'] = df['text'].apply(lambda x: get_vec(x))

Por el momento no podemos utilizar word2vec, pues se trata de textos demasiado largos y superamos le limite permitido por el algoritmo.
Por lo que seguiremos investigando otra cosa

# Optimización de SVC
Tras probar varios métodos y ver diferentes técnicas que realmente no se pueden aplicar a nuestro caso al ser textos demasiado grandes. Hemos optado por quedarnos con SVM. Por lo que, vamos a optimizar nuestro modelo.

## Paso 1: Optimización de TFIDF()
En primer lugar vamos a optimizar el tfidf y luego seguiremos con nuestro Linear SVC, ya habiendole aplicado a esta optimización PCA.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data["text"], y_, test_size = 0.2, random_state = 0)

In [ ]:
pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', OneVsRestClassifier(LinearSVC(C=1.5, penalty='l1',dual=False)))
])

In [ ]:
hyperparameters = {
     'tfidf__max_df': (0.5, 1.0),
     'tfidf__ngram_range': ((1,1), (1,2)),
     'tfidf__use_idf': (True, False),
     'tfidf__analyzer': ('word', 'char', 'char_wb')
}

In [ ]:
clf = GridSearchCV(pipe, hyperparameters, cv = 5)

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
clf.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer(use_idf=False)),
                ('svm',
                 OneVsRestClassifier(estimator=LinearSVC(C=1.5, dual=False,
                                                         penalty='l1')))])

In [ ]:
clf.best_params_

{'tfidf__analyzer': 'word',
 'tfidf__max_df': 1.0,
 'tfidf__ngram_range': (1, 1),
 'tfidf__use_idf': False}

In [ ]:
clf.best_score_

0.18926553672316385

In [ ]:
pkl.dump(clf, open('/content/drive/MyDrive/libros/model_libros.pkl', 'wb'))

In [ ]:
y_pred=clf.predict(x_test)
j_score(y_test, y_pred)

37.61261261261261

## Paso 2: Aplicación de TFIDF y PCA
Una vez aquí, vamos a aplicar a nuestros datos en crudo, TFIDF y luego usaremos la mejor versión de PCA, antes de aplicar la optimización de SVM.

In [ ]:
data_x=data["text"]
data_y=y_

In [ ]:
# Aplicamos TFIDF con los parámetros que obtuvimos de la optimización previa.
tfidf=TfidfVectorizer(analyzer="word", max_df=1.0, ngram_range=(1,1), use_idf=False)
data_x=tfidf.fit_transform(data_x)

print("shape of x", data_x.shape)
print("shape of y", data_y.shape)

shape of x (370, 428732)
shape of y (370, 8)


In [ ]:
# APLICACIÓN DE PCA
data_x_array=data_x.toarray()
data_x_pca=pd.DataFrame(data_x_array)
# Como se vió al principio aplicaremos una varianza explicativa del 90%
min_var = 0.9
pca_var= PCA(n_components=min_var, svd_solver='full')

X_pca = pca_var.fit_transform(data_x_pca)

pca_var= PCA(n_components=min_var, svd_solver='full')

X_pca2 = pca_var.fit(data_x_pca)

print('Nº de componentes:', len(X_pca2.components_))
print('Ratio de varianza explicada:')
print(X_pca2.explained_variance_ratio_)

Nº de componentes: 220
Ratio de varianza explicada:
[0.1413853  0.04316516 0.04034279 0.02607837 0.02447925 0.01744034
 0.01629088 0.01433617 0.01231211 0.01186816 0.00993669 0.00948618
 0.00847198 0.00840352 0.00810006 0.00789272 0.00729348 0.00701183
 0.00663332 0.00607232 0.00600391 0.00584406 0.00576609 0.00560865
 0.00534868 0.00530455 0.00519995 0.00509313 0.0049705  0.00486676
 0.00476702 0.0046979  0.00456548 0.00446508 0.00437419 0.00434308
 0.00428057 0.00423406 0.00420833 0.00413981 0.00400939 0.00395596
 0.00389156 0.00388216 0.00377178 0.00373012 0.00367991 0.00365872
 0.00359095 0.00350131 0.00344499 0.0033955  0.00332709 0.00329398
 0.00328122 0.00325171 0.00321473 0.00319769 0.00316785 0.00313389
 0.00311213 0.00309338 0.0030485  0.00304224 0.00298777 0.00297695
 0.00295325 0.00293188 0.00292137 0.00286193 0.00283434 0.00280564
 0.00276308 0.00274382 0.0027413  0.00271652 0.00268423 0.00268328
 0.0026657  0.0026476  0.00262001 0.00260329 0.00257808 0.00256038
 0.0025502

## Optimización del Linear SVC

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_pca, data_y, test_size = 0.2, random_state = 0)

In [ ]:
model = OneVsRestClassifier(estimator=LinearSVC(dual=False), n_jobs=-1)

hyperparameters = {
     'estimator__penalty': ["l1","l2"],
     'estimator__C': np.arange(0.01,100,10),
     'estimator__loss': ["hinge","squared_hinge"],
     'estimator__multi_class':["ovr","crammer_singer"]
     
}

clf=GridSearchCV(estimator=model,param_grid= hyperparameters, cv=5)

In [ ]:
clf.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
47 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py", line 428, in _process_worker
    r = call_item()
  File "/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py", line 275, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.7/dist-packages/joblib/_parallel_backends.py", line 620, in __call__
    return self.func(

GridSearchCV(cv=5,
             estimator=OneVsRestClassifier(estimator=LinearSVC(dual=False),
                                           n_jobs=-1),
             param_grid={'estimator__C': array([1.000e-02, 1.001e+01, 2.001e+01, 3.001e+01, 4.001e+01, 5.001e+01,
       6.001e+01, 7.001e+01, 8.001e+01, 9.001e+01]),
                         'estimator__loss': ['hinge', 'squared_hinge'],
                         'estimator__multi_class': ['ovr', 'crammer_singer'],
                         'estimator__penalty': ['l1', 'l2']})

In [ ]:
clf.best_estimator_

OneVsRestClassifier(estimator=LinearSVC(C=2.5, dual=False, loss='hinge',
                                        multi_class='crammer_singer',
                                        penalty='l1'),
                    n_jobs=-1)

In [ ]:
clf.best_params_

{'estimator__C': 2.5,
 'estimator__loss': 'hinge',
 'estimator__multi_class': 'crammer_singer',
 'estimator__penalty': 'l1'}

In [ ]:
clf.best_score_

0.337683615819209

In [ ]:
pkl.dump(clf, open('/content/drive/MyDrive/libros/model_libros_final.pkl', 'wb'))

In [ ]:
y_pred=clf.predict(x_test)
j_score(y_test, y_pred)

52.477477477477485

In [ ]:
# En algunos casos clasifica solo uno de los géneros, en otros acierta todo y en otros no es capaz de realizar una clasificación en ninguno de los géneros.
for i in range(len(y_test)):
  print("predicción->",y_pred[i], "Real->", y_test.iloc[i].tolist())

predicción-> [0 1 0 0 0 0 0 0] Real-> [0, 1, 0, 0, 0, 1, 0, 0]
predicción-> [0 0 0 0 0 0 0 0] Real-> [0, 1, 0, 0, 0, 0, 0, 0]
predicción-> [0 0 0 0 0 0 1 0] Real-> [0, 0, 0, 0, 0, 0, 1, 0]
predicción-> [0 0 0 1 0 0 0 0] Real-> [0, 0, 0, 1, 0, 0, 0, 0]
predicción-> [0 0 0 0 0 0 0 0] Real-> [0, 0, 0, 0, 1, 0, 1, 0]
predicción-> [0 1 0 0 1 0 0 0] Real-> [0, 1, 0, 0, 1, 0, 0, 0]
predicción-> [0 0 0 0 0 0 0 1] Real-> [0, 0, 0, 0, 0, 0, 0, 1]
predicción-> [0 0 0 0 0 0 0 0] Real-> [0, 0, 1, 0, 0, 1, 0, 0]
predicción-> [1 0 0 0 0 0 0 0] Real-> [1, 0, 0, 0, 0, 0, 0, 0]
predicción-> [1 0 0 1 0 0 0 0] Real-> [0, 0, 0, 1, 0, 0, 0, 0]
predicción-> [1 0 0 0 0 0 0 0] Real-> [1, 0, 0, 0, 0, 0, 0, 0]
predicción-> [0 0 0 0 0 0 0 0] Real-> [0, 0, 0, 0, 0, 0, 1, 0]
predicción-> [0 0 0 0 0 0 0 0] Real-> [0, 0, 0, 0, 1, 0, 0, 0]
predicción-> [0 0 0 0 0 0 0 0] Real-> [0, 0, 0, 0, 1, 0, 0, 0]
predicción-> [0 0 0 0 1 0 0 0] Real-> [0, 0, 0, 1, 0, 0, 0, 0]
predicción-> [0 0 0 0 1 0 0 0] Real-> [0, 0, 0, 1, 1, 0

# Funcionamiento del modelo

In [ ]:
from pickleshare import pickle
with open('/content/drive/MyDrive/libros/model_libros_final.pkl', "rb") as file:
  modelo=pickle.load(file)

In [ ]:
def get_clean(x):
    x = str(x).lower().replace('\\', ' ').replace('\r\n', ' '  )
    x = kgp.cont_exp(x)
    x = kgp.remove_emails(x)
    x = kgp.remove_urls(x)
    x = kgp.remove_html_tags(x)
    x = kgp.remove_accented_chars(x)
    x = kgp.remove_special_chars(x)
    x = kgp.remove_dups_char(x)


    return x

In [ ]:
def clasificador_libros(url, model):
  response=requests.get(url)
  texto=response.text
  x=get_clean(texto)
  x= " ".join([t for t in x.split() if t not in stopwords])
  x=pd.Series([x])
  x_tfidf=tfidf.transform(x)
  x_tfidf=x_tfidf.toarray()
  x_pca=pca_var.transform(x_tfidf)
  prediccion= model.predict(x_pca)
  clasificador=[]
  if prediccion[0][0]==1:
    clasificador.append("humor")
  if prediccion[0][1]==1:
    clasificador.append("amor")
  if prediccion[0][2]==1:
    clasificador.append("ciencia ficción")
  if prediccion[0][3]==1:
    clasificador.append("aventura")
  if prediccion[0][4]==1:
    clasificador.append("histórica")
  if prediccion[0][5]==1:
    clasificador.append("psicológica")
  if prediccion[0][6]==1:
    clasificador.append("guerra")
  if prediccion[0][7]==1:
    clasificador.append("terror")
  print("estamos ante una novela de ->", clasificador)
  return prediccion, clasificador

In [ ]:
prediccion = clasificador_libros("https://www.gutenberg.org/ebooks/29954.txt.utf-8", modelo)

estamos ante una novela de -> ['ciencia ficción', 'terror']
